In [1]:
# https://github.com/Neerajj9/Text-Detection-using-Yolo-Algorithm-in-keras-tensorflow/blob/master/Preprocess.py

In [2]:
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from PIL import Image
import pytesseract

import matplotlib.pyplot as plt

In [3]:
pwd

'/home/scar3crow/Dropbox/WorkStation-Subrata/python'

In [4]:
cd models

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models


In [7]:
ls

AUTHORS     community/       ISSUES.md  official/  research/
CODEOWNERS  CONTRIBUTING.md  LICENSE    README.md


In [8]:
cd research

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models/research


In [9]:
ls

a3c_blogpost/                      lm_1b/
adversarial_crypto/                lm_commonsense/
adversarial_logit_pairing/         lstm_object_detection/
adversarial_text/                  marco/
adv_imagenet_models/               maskgan/
attention_ocr/                     namignizer/
audioset/                          neural_gpu/
autoaugment/                       neural_programmer/
autoencoder/                       next_frame_prediction/
brain_coder/                       nst_blogpost/
cognitive_mapping_and_planning/    object_detection/
cognitive_planning/                pcl_rl/
compression/                       ptn/
cvt_text/                          qa_kg/
deep_contextual_bandits/           README.md
deeplab/                           real_nvp/
deep_speech/                       rebar/
delf/                              sentiment_analysis/
domain_adaptation/                 seq2species/
efficient-hrl/                     setup.py
feelvos/                           skip_thoughts/
f

In [10]:
cd object_detection

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models/research/object_detection


In [12]:
ls

anchor_generators/                   legacy/
box_coders/                          matchers/
builders/                            meta_architectures/
CONTRIBUTING.md                      metrics/
core/                                model_hparams.py
data/                                model_lib.py
data_decoders/                       model_lib_test.py
dataset_tools/                       model_lib_v2.py
dockerfiles/                         model_lib_v2_test.py
eval_util.py                         model_main.py
eval_util_test.py                    models/
exporter.py                          model_tpu_main.py
exporter_test.py                     object_detection_tutorial.ipynb
export_inference_graph.py            predictors/
export_tflite_ssd_graph_lib.py       protos/
export_tflite_ssd_graph_lib_test.py  README.md
export_tflite_ssd_graph.py           samples/
g3doc/                               test_data/
inference/                           test_images/
__init__.py                   

In [43]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from utils import *

from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import *

from keras.applications import MobileNetV2
from keras.models import Model
from keras.models import model_from_json


In [14]:
# Making a list of image path

inv_directory = '/home/scar3crow/Downloads/INVOICES'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_image = ['/home/scar3crow/Downloads/INVOICES/{}'.format(i) for i in os.listdir(inv_directory)] # making the list
inv_image.sort() # Sorting the list

print('Number of images = ', len(inv_image))
inv_image[50]

Number of images =  66


'/home/scar3crow/Downloads/INVOICES/53.jpg'

In [15]:
# Check sizes of exiting images :

# Create a Dataframe with image id and height(row) and width(column):

rows = []
columns = []
image_sl = []
df = pd.DataFrame()

for i in range(len(inv_image)):
    image = cv2.imread(inv_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df.insert(loc=0, column='image_serial', value=image_num)
df.insert(loc=1, column='rows', value=row_values)
df.insert(loc=2, column='columns', value=col_values)

df.head()


,image_serial,rows,columns
0,/home/scar3crow/Downloads/INVOICES/1.jpg,768,554
1,/home/scar3crow/Downloads/INVOICES/10.jpg,768,554
2,/home/scar3crow/Downloads/INVOICES/11.jpg,768,554
3,/home/scar3crow/Downloads/INVOICES/12.jpg,768,554
4,/home/scar3crow/Downloads/INVOICES/13.jpg,768,554


In [16]:
# Finding minimimum sizes :

i = df[['rows']].idxmin()
j = df[['columns']].idxmin()

print(i)
print(j)

print(df.loc[i])
print(df.loc[j])

rows    0
dtype: int64
columns    0
dtype: int64
                               image_serial  rows  columns
0  /home/scar3crow/Downloads/INVOICES/1.jpg   768      554
                               image_serial  rows  columns
0  /home/scar3crow/Downloads/INVOICES/1.jpg   768      554


In [17]:
# Resize to 768 X 554

for i in range(len(df)):
    
    original_image = cv2.imread(df['image_serial'][i])
    resized_image = cv2.resize(original_image, (554, 768)) ## Note width(column), height(row) format in resize
    cv2.imwrite(df['image_serial'][i], resized_image) ## writting or over-writting on the old image

i_width = 554
i_height = 768

In [18]:
# Loading output of VGG Image Annotation tool and create a dataframe

r_data = pd.read_csv('/home/scar3crow/Downloads/invoice_data_1/via_region_data.csv')
num_obj = r_data['region_count'][0] # number of objects in each photo
r_data.drop(r_data.columns[[1, 2, 3, 4]], axis=1, inplace=True) # reduce unnecessary columns
r_data.sort_values(by=['#filename'], ascending=True) # Sorting based on image-id
num_images = r_data["#filename"].nunique() # Find out number of unique images

print('Number of classes = ', num_obj)
print('Number of unique images = ', num_images)
r_data.head()

Number of classes =  7
Number of unique images =  30


,#filename,region_shape_attributes,region_attributes
0,30.jpg,"{""name"":""rect"",""x"":91,""y"":125,""width"":394,""hei...","{""text"":""vendor""}"
1,30.jpg,"{""name"":""rect"",""x"":79,""y"":175,""width"":426,""hei...","{""text"":""buyer""}"
2,30.jpg,"{""name"":""rect"",""x"":38,""y"":83,""width"":136,""heig...","{""text"":""invoice""}"
3,30.jpg,"{""name"":""rect"",""x"":412,""y"":88,""width"":76,""heig...","{""text"":""inv_date""}"
4,30.jpg,"{""name"":""rect"",""x"":94,""y"":253,""width"":74,""heig...","{""text"":""po""}"


In [19]:
# Making a dataframe for Image_id, x, y, width, height and class

x = []
y = []
width = []
height = []
obj_class = []


for i in range(len(r_data)):
    
    r_size = r_data.values[i, 1][1:(len(r_data.values[i, 1])-1)]
    r_size_par = r_size.split(",")
    
    x.append(int("".join(filter(str.isdigit, r_size_par[1]))))
    y.append(int("".join(filter(str.isdigit, r_size_par[2]))))
    width.append(int("".join(filter(str.isdigit, r_size_par[3]))))
    height.append(int("".join(filter(str.isdigit, r_size_par[4]))))
    
    r_attribs = r_data.values[i, 2][1:(len(r_data.values[i, 2])-1)]
    r_attribs_par = r_attribs.split(':')[1]
    obj_class.append(r_attribs_par[1:(len(r_attribs_par)-1)])
    
x_values = pd.Series(x)
y_values = pd.Series(y)
width_values = pd.Series(width)
height_values = pd.Series(height)
class_values = pd.Series(obj_class)

r_data.insert(loc=1, column='x', value=x_values)
r_data.insert(loc=2, column='y', value=y_values)
r_data.insert(loc=3, column='width', value=width_values)
r_data.insert(loc=4, column='height', value=height_values)
r_data.insert(loc=5, column='obj_class', value=class_values)

r_data.insert(loc=8, column='image_width', value=i_width)
r_data.insert(loc=9, column='image_height', value=i_height)


r_data.drop(r_data.columns[[6, 7]], axis=1, inplace=True) # reduce unnecessary columns

r_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name

r_data.head()

,img_id,x,y,width,height,obj_class,image_width,image_height
0,30.jpg,91,125,394,45,vendor,554,768
1,30.jpg,79,175,426,81,buyer,554,768
2,30.jpg,38,83,136,26,invoice,554,768
3,30.jpg,412,88,76,23,inv_date,554,768
4,30.jpg,94,253,74,19,po,554,768


In [20]:
print('Number of unique images = ', r_data['img_id'].nunique())  # print total no, of unique images

print('Number of classes in diff. categories = ', r_data['obj_class'].value_counts()) 


Number of unique images =  30
Number of classes in diff. categories =  po              30
buyer           30
invoice         30
vendor          30
inv_date        30
totals          29
item_detail     28
tem_detail       1
item_details     1
total            1
Name: obj_class, dtype: int64


In [21]:
# Correcting above wrong spelling of object classes and rechecking

id_1 = r_data.index[r_data['obj_class'] == 'tem_detail'] # Finding the index
id_2 = r_data.index[r_data['obj_class'] == 'total'] # Finding the index
id_3 = r_data.index[r_data['obj_class'] == 'item_details'] # Finding the index

r_data.at[id_1, 'obj_class'] = 'item_detail' # writing the correct spelling 
r_data.at[id_2, 'obj_class'] = 'totals' # # writing the correct spelling
r_data.at[id_3, 'obj_class'] = 'item_detail' # # writing the correct spelling

print('Number of unique images = ', r_data['img_id'].nunique())  # print total no, of unique images
print('Number of unique classes = ', r_data['obj_class'].nunique())
print('Number of classes in diff. categories = ', r_data['obj_class'].value_counts()) 


Number of unique images =  30
Number of unique classes =  7
Number of classes in diff. categories =  item_detail    30
po             30
buyer          30
invoice        30
totals         30
vendor         30
inv_date       30
Name: obj_class, dtype: int64


In [22]:
## Creating train images by cropping original images as per x, y, width, height of bbox created in VGG I.A. tool

my_class_image = [] # List of class images with proper id
kount = 0

for i in range(len(inv_image)):
    
    id = inv_image[i].split('/')[-1] # finding the image_id
    indices = r_data.index[r_data['img_id'] == id].tolist() # making the list of indices of r_data where same image_id exists
    
    if len(indices) != 0:
        
        image = cv2.imread(inv_image[i]) ## Loading image
    
        kount = kount + 1
        
        # Cropping, making a training image list and saving training images in 'train_invoice' folder
    
        for j in indices:
        
            image_crop = image[r_data['y'][j]:r_data['y'][j]+r_data['height'][j], r_data['x'][j]:r_data['x'][j]+r_data['width'][j]]
        
            class_sl_id = '/home/scar3crow/Downloads/train_invoice/' + id.split('.')[0] + '-' + str(r_data['obj_class'][j]) + str(kount) + '.jpg'
        
            my_class_image.append(class_sl_id)
            
            cv2.imwrite(class_sl_id, image_crop)

In [23]:
# Read image, calculate ratio with target size, resize the image to target size :

i_path = '/home/scar3crow/Downloads/INVOICES/' + r_data['img_id'][0] # set image path

i_mage = cv2.imread(i_path) # read images
    
x_ratio = 512/i_mage.shape[1] # calculate ratio
y_ratio = 512/i_mage.shape[0] # calculate ratio

print(x_ratio, y_ratio)
print(i_mage.shape[1], i_mage.shape[0])

x = cv2.resize(i_mage,(512,512)) # resize the image

print(x.shape[1], x.shape[0])

0.924187725631769 0.6666666666666666
554 768
512 512


In [24]:
categories = r_data['obj_class'].unique() # make a list of unique images

values = np.array(categories)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_encoded = onehot_encoder.fit_transform(integer_encoded)

print(categories)                               
print(y_encoded)


['vendor' 'buyer' 'invoice' 'inv_date' 'po' 'item_detail' 'totals']
[[0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]


In [25]:
# Preparing Input(X) and Target(Y) file for training :

# Variable Definition

channels = 3
anchors = 1
classes = 7 # vendor, buyer, invoice, inv_date, po, item_detail, totals
info = 5 + classes    # pc, x, y, h, w, and class probabilities

grid_y_axis = 16  # each image is to be segmented to 16 x 16 grid
grid_x_axis = 16  # # each image is to be segmented to 16 x 16 grid

img_w = 512 # target sizes as per MobileNetV2
img_h = 512 #target sizes as per MobileNetV2

# Start :

X_final = [] # X_final list to convert to np array later
Y_final = [] # Y_final list to convert to np array later

image_list = r_data['img_id'].unique() # make a list of unique images

for i in image_list:
    
    image_path = '/home/scar3crow/Downloads/INVOICES/' + i
    
    x = cv2.imread(image_path)
    x_ratio = img_w / x.shape[1]
    y_ratio = img_h / x.shape[0]
    img = cv2.resize(x,(512,512))
    
    X_final.append(img)
    
    Y = np.zeros((grid_y_axis,grid_x_axis,1,info))
    
    r_data_slice = r_data.loc[r_data['img_id'].isin([i])].reset_index(drop=True) # slicing as per current image id
    
    for j in range(len(r_data_slice)):
        
        xmin = r_data_slice['x'][j] * x_ratio
        ymin = r_data_slice['y'][j] * y_ratio
        
        xmax = (r_data_slice['x'][j] + r_data_slice['width'][j]) * x_ratio
        ymax = (r_data_slice['y'][j] + r_data_slice['height'][j]) * y_ratio
        
        obj_class = r_data_slice['obj_class'][j]
        
        w = (xmax - xmin)/img_w
        h = (ymax - ymin)/img_h

        x = (xmin + (xmax-xmin)/2)/img_w
        y = (ymin + (ymax-ymin)/2)/img_h
        
        x = x * grid_y_axis
        y = y * grid_x_axis
        
        Y[int(y),int(x),0,0] = 1
        Y[int(y),int(x),0,1] = x - int(x)
        Y[int(y),int(x),0,2] = y - int(y)
        Y[int(y),int(x),0,3] = w
        Y[int(y),int(x),0,4] = h
        
        balance_Y = y_encoded[categories.tolist().index(obj_class), :]
        
        Y[int(y),int(x),0,5] = balance_Y[0]
        Y[int(y),int(x),0,6] = balance_Y[1]
        Y[int(y),int(x),0,7] = balance_Y[2]
        Y[int(y),int(x),0,8] = balance_Y[3]
        Y[int(y),int(x),0,9] = balance_Y[4]
        Y[int(y),int(x),0,10] = balance_Y[5]
        Y[int(y),int(x),0,11] = balance_Y[6]
        
    Y_final.append(Y)
    
X = np.array(X_final)
X_final = []
Y = np.array(Y_final)
Y_final = []

X = (X - 127.5)/127.5  # X normalising since pixels vary from 0 to 255
    
np.save('/home/scar3crow/Downloads/Data1/X.npy',X)
np.save('/home/scar3crow/Downloads/Data1/Y.npy',Y)
   

In [26]:
print(X.shape)
print(Y.shape)
X.shape[:2]

(30, 512, 512, 3)
(30, 16, 16, 1, 12)


(30, 512)

In [27]:
# output shape = (1, a, b, 1, 12)

def decode_to_boxes(output , ht , wd):   # output = model.predict, ht = img_h = 512, wd = img_w = 512
    #output : (x,x,1,12)
    #x,y,h,w

    img_ht = ht
    img_wd = wd
    threshold = 0.5
    grid_y_axis, grid_x_axis = output.shape[:2]
    final_boxes = []
    scores_top = []
    scores_class = []

    for i in range(grid_h):
        for j in range(grid_w):
            if output[i,j,0,0] > threshold:

                temp_1 = output[i, j, 0, 1:5]
                temp_2 = output[i, j, 0, 5:12]
                
                x_unit = ((j + (temp_1[0])) / grid_w) * img_wd
                y_unit = ((i + (temp_1[1])) / grid_h) * img_ht
                
                w_unit = temp_1[2] * img_wd
                h_unit = temp_1[3] * img_ht
                
                xmax = x_unit + w_unit * 0.5
                xmin = x_unit - w_unit * 0.5
                
                ymax = y_unit + h_unit * 0.5
                ymin = y_unit - h_unit * 0.5
                
                xmax_original = xmax / x_ratio
                xmin_original = xmin / x_ratio
                
                ymax_original = ymax / y_ratio
                ymin_original = ymin / y_ratio
                
                class_index = np.argmax(temp_2)
                obj_class = sorted(categories.tolist())[class_index]
                
                
                final_boxes.append([xmin_original, ymin_original, xmax_original, ymax_original], obj_class)
                scores_top.append(output[i,j,0,0], obj_class)
                scores_class.append(output[i, j, 0, (5+class_index)], obj_class)
    
    return final_boxes,scores_top, scores_class


In [28]:
def iou(box1,box2):

    x1 = max(box1[0],box2[0])
    x2 = min(box1[2],box2[2])
    y1 = max(box1[1] ,box2[1])
    y2 = min(box1[3],box2[3])
    
    inter = (x2 - x1)*(y2 - y1)
    
    area1 = (box1[2] - box1[0])*(box1[3] - box1[1])
    area2 = (box2[2] - box2[0])*(box2[3] - box2[1])
    fin_area = area1 + area2 - inter
        
    iou = inter/fin_area
    
    return iou

In [49]:
#define loss function

def yolo_loss_func(y_true,y_pred):
    
    # y_true : 16,16,1,12
    # y_pred : 16,16,1,12
    
    l_coords = 5.0
    l_noob = 0.5
    
    coords = y_true[:,:,:,:,0] * l_coords  # If the grid has a object - weitage increased by l_coords(5 times)
    noobs = (-1*(y_true[:,:,:,:,0] - 1)*l_noob) # If the grid does not have an object - penalised
    p_pred = y_pred[:,:,:,:,0]
    p_true = y_true[:,:,:,:,0]
    x_true = y_true[:,:,:,:,1]
    x_pred = y_pred[:,:,:,:,1]
    yy_true = y_true[:,:,:,:,2]
    yy_pred = y_pred[:,:,:,:,2]
    w_true = y_true[:,:,:,:,3]
    w_pred = y_pred[:,:,:,:,3]
    h_true = y_true[:,:,:,:,4]
    h_pred = y_pred[:,:,:,:,4]
    
    class_true = y_true[:,:,:,:,5:12]
    class_pred = y_pred[:,:,:,:,5:12]
    
    
    p_loss_absent = K.sum(K.square(p_pred - p_true)*noobs)
    p_loss_present = K.sum(K.square(p_pred - p_true)) * (coords / l_coords) # (coords/l_coords) added by me to 
                                                                            # exactly follow yolo loss formula
    x_loss = K.sum(K.square(x_pred - x_true)*coords)
    yy_loss = K.sum(K.square(yy_pred - yy_true)*coords)
    xy_loss = x_loss + yy_loss
    
    w_loss = K.sum(K.square(K.sqrt(w_pred) - K.sqrt(w_true))*coords)
    h_loss = K.sum(K.square(K.sqrt(h_pred) - K.sqrt(h_true))*coords)
    wh_loss = w_loss + h_loss
    
    class_loss = K.sum(K.sum(K.square(class_pred - class_true)) * (coords / l_coords)) # added by me
    
    
    loss = p_loss_absent + p_loss_present + xy_loss + wh_loss + class_loss
    
    return loss

In [30]:
def non_max(boxes , scores , iou_num):

    scores_sort = scores.argsort().tolist()
    keep = []
    
    while(len(scores_sort)):
        
        index = scores_sort.pop()
        keep.append(index)
        
        if(len(scores_sort) == 0):
            break
    
        iou_res = []
    
        for i in scores_sort:
            iou_res.append(iou(boxes[index] , boxes[i]))
        
        iou_res = np.array(iou_res)
        filtered_indexes = set((iou_res > iou_num).nonzero()[0])

        scores_sort = [v for (i,v) in enumerate(scores_sort) if i not in filtered_indexes]
    
    final = []
    
    for i in keep:
        final.append(boxes[i])
    
    return final


def decode(output , ht , wd , iou):
    
    
    boxes , scores = decode_to_boxes(output ,ht ,wd)
    
    
    boxes = non_max(boxes,np.array(scores) , iou)
    
    
    return boxes
    


In [31]:
def save_model(model):
    model_json = model.to_json()
    with open("/home/scar3crow/Downloads/text_detect_model.json", "w") as json_file:
        json_file.write(model_json)

        
        
def load_model(strr):        
    json_file = open(strr, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model

In [32]:
#import data
#X and Y numpy arrays are created using the Prepocess.py file
X = np.load('/home/scar3crow/Downloads/Data1/X.npy')
Y = np.load('/home/scar3crow/Downloads/Data1/Y.npy')


In [33]:
print(X.shape , Y.shape)

(30, 512, 512, 3) (30, 16, 16, 1, 12)


In [34]:
X_train , X_val , Y_train , Y_val  = train_test_split(X,Y,train_size = 0.75 , shuffle = True)
X = []
Y = []

/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [35]:
def yolo_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    conv = Conv2D(512,(3,3) , activation='relu' , padding='same')(last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(128,(5,5) , activation='relu' , padding='same')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(12,(3,3) , activation='relu' , padding='same')(lr)
    
    final = Reshape((grid_y_axis,grid_x_axis,anchors,info))(conv)
    
    model = Model(inp,final)
    
    return model


In [36]:
#define utils

#optimizer
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

#checkpoint
checkpoint = ModelCheckpoint('model1/text_detect.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min',period=1)


In [38]:
input_size = (img_h,img_w,channels)

yolo_invoice_model = yolo_model(input_size)

print(yolo_invoice_model.summary())

save_model(yolo_invoice_model)

/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [39]:
yolo_invoice_model.load_weights('/home/scar3crow/Downloads/text_detect.h5')    

ValueError: Shapes (5, 5, 512, 128) and (128, 512, 3, 3) are incompatible

In [50]:


yolo_invoice_model.compile(loss=yolo_loss_func , optimizer=opt , metrics = ['accuracy'])

In [51]:
def yolo_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    conv = Conv2D(512,(3,3) , activation='relu' , padding='same')(last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
#    conv = Conv2D(128,(5,5) , activation='relu' , padding='same')(lr)
#    conv = Dropout(0.4)(conv)
#    bn = BatchNormalization()(conv)
#    lr = LeakyReLU(alpha=0.1)(bn)
    
    
#    conv = Conv2D(12,(3,3) , activation='relu' , padding='same')(lr)
    
#    final = Reshape((grid_y_axis,grid_x_axis,anchors,info))(conv)
    
#    model = Model(inp,final)
    
#    return model


    conv = Conv2D(128,(3,3) , activation='relu' , padding='same')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(5,(3,3) , activation='relu' , padding='same')(lr)
    
    final = Reshape((grid_y_axis,grid_x_axis,1,5))(conv)
    
    model = Model(inp,final)
    
    return model

In [52]:
input_size = (img_h,img_w,channels)

yolo_invoice_model_1 = yolo_model(input_size)

print(yolo_invoice_model_1.summary())

save_model(yolo_invoice_model_1)

/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [53]:
yolo_invoice_model_1.load_weights('/home/scar3crow/Downloads/text_detect.h5')

In [59]:
input_tensor = Input(shape=(512, 512, 3))
new_model = MobileNetV2(input_tensor = input_tensor, weights='imagenet')
new_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [89]:
mmm = MobileNetV2(input_tensor = input_tensor, include_top = False, weights='imagenet')
mmm.summary()

/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [98]:
def my_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    last_3 = Flatten()(last_layer)
    last_2 = Dense(640, activation = 'relu')(last_3)
    last_1 = Dense(320, activation = 'relu')(last_2)
    last_0 = Dense(7, activation = 'softmax')(last_1)
        
    model = Model(inp,last_0)
    
    return model


In [61]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
new_model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [72]:
image_directory = '/home/scar3crow/Downloads/train_invoice'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_crop_image = ['/home/scar3crow/Downloads/train_invoice/{}'.format(i) for i in os.listdir(image_directory)] # making the list
inv_crop_image.sort() # Sorting the list

print('Number of images = ', len(inv_crop_image))
inv_crop_image[50]

Number of images =  210


'/home/scar3crow/Downloads/train_invoice/16-inv_date8.jpg'

In [83]:
aa = inv_crop_image[50].split('/')
bb = aa[-1].split('-')[-1].split('.')

cc = ''.join(i for i in bb[0] if not i.isdigit())

y_encoded[categories.tolist().index(cc), :]

array([0., 1., 0., 0., 0., 0., 0.])

In [84]:
# Preparing Input(X) and Target(Y) file for training :

# Variable Definition

img_w = 512 # target sizes as per MobileNetV2
img_h = 512 #target sizes as per MobileNetV2

# Start :

X_final = [] # X_final list to convert to np array later
Y_final = [] # Y_final list to convert to np array later

# image_list = r_data['img_id'].unique() # make a list of unique images

for i in range(len(inv_crop_image)):
    
    image_path = inv_crop_image[i]
    
    x = cv2.imread(image_path)
    x_ratio = img_w / x.shape[1]
    y_ratio = img_h / x.shape[0]
    img = cv2.resize(x,(512,512))
    
    X_final.append(img)
    
    Y = np.zeros((7))
    
    y_1 = inv_crop_image[i].split('/')
    y_2 = y_1[-1].split('-')[-1].split('.')
    y_3 = ''.join(i for i in y_2[0] if not i.isdigit())
    
    Y =  y_encoded[categories.tolist().index(y_3), :]
                    
    Y_final.append(Y)
    
X = np.array(X_final)
X_final = []
Y = np.array(Y_final)
Y_final = []

X = (X - 127.5)/127.5  # X normalising since pixels vary from 0 to 255
    
np.save('/home/scar3crow/Downloads/Data1/X_1.npy',X)
np.save('/home/scar3crow/Downloads/Data1/Y_1.npy',Y)


In [85]:
X.shape

(210, 512, 512, 3)

In [86]:
Y.shape

(210, 7)

In [87]:
#import data
#X and Y numpy arrays are created using the Prepocess.py file
X = np.load('/home/scar3crow/Downloads/Data1/X_1.npy')
Y = np.load('/home/scar3crow/Downloads/Data1/Y_1.npy')

X_train , X_val , Y_train , Y_val  = train_test_split(X,Y,train_size = 0.75 , shuffle = True)
X = []
Y = []
X_train.shape

/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(157, 512, 512, 3)

In [101]:
Y_train.shape

(157, 7)

In [88]:
new_model.fit(X_train ,Y_train ,epochs= 30 ,batch_size = 4, validation_data=(X_val,Y_val))


ValueError: Error when checking target: expected Logits to have shape (1000,) but got array with shape (7,)

In [99]:
input_size = (img_h,img_w,channels)

my_invoice_model = my_model(input_size)

print(my_invoice_model.summary())


/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [100]:
my_invoice_model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
my_invoice_model.fit(X_train ,Y_train ,epochs= 30 ,batch_size = 4, validation_data=(X_val,Y_val))


Train on 157 samples, validate on 53 samples
Epoch 1/30
 16/157 [==>...........................] - ETA: 1:42:11 - loss: 10.6989 - accuracy: 0.2500